In [1]:
import tensorflow.keras as keras
from tensorflow.keras import backend as K
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

%matplotlib inline

In [2]:
! unzip -o cat_dataset.zip

Archive:  cat_dataset.zip
  inflating: cat_dataset/cat_data.csv  
  inflating: cat_dataset/cat_images/cat_0.jpg  
  inflating: cat_dataset/cat_images/cat_1.jpg  
  inflating: cat_dataset/cat_images/cat_10.jpg  
  inflating: cat_dataset/cat_images/cat_100.jpg  
  inflating: cat_dataset/cat_images/cat_1000.jpg  
  inflating: cat_dataset/cat_images/cat_1001.jpg  
  inflating: cat_dataset/cat_images/cat_1002.jpg  
  inflating: cat_dataset/cat_images/cat_1003.jpg  
  inflating: cat_dataset/cat_images/cat_1004.jpg  
  inflating: cat_dataset/cat_images/cat_1005.jpg  
  inflating: cat_dataset/cat_images/cat_1006.jpg  
  inflating: cat_dataset/cat_images/cat_1007.jpg  
  inflating: cat_dataset/cat_images/cat_1008.jpg  
  inflating: cat_dataset/cat_images/cat_1009.jpg  
  inflating: cat_dataset/cat_images/cat_101.jpg  
  inflating: cat_dataset/cat_images/cat_1010.jpg  
  inflating: cat_dataset/cat_images/cat_1011.jpg  
  inflating: cat_dataset/cat_images/cat_1012.jpg  
  inflating: cat_dataset/c

In [3]:
def grab_image(img_path):
    img = image.load_img(img_path, target_size=(100, 100))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x
X = np.concatenate([grab_image('./cat_dataset/cat_images/cat_%d.jpg'%(i))for i in range(3000)])
X.shape

Y = np.loadtxt('./cat_dataset/cat_data.csv')

x = keras.layers.Input(X.shape[1:])
y = x
y = keras.layers.Conv2D(64, kernel_size=(4,4), activation='relu')(y)
#y = keras.layers.Conv2D(128, kernel_size=(4,4), activation='relu')(y)
y = keras.layers.MaxPooling2D(pool_size=(4,4))(y)
y = keras.layers.Dropout(0.25)(y)
y = keras.layers.Flatten()(y)
y = keras.layers.Dense(128, activation='relu')(y)
y = keras.layers.Dropout(0.5)(y)
# Output Logits (10)
y = keras.layers.Dense(len(np.unique(Y)))(y)
model = keras.Model(x,y)
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
 optimizer=keras.optimizers.Adam(),
 metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.summary()

data_generator = keras.preprocessing.image.ImageDataGenerator(
 width_shift_range=0.5,
 height_shift_range=0.5,
 horizontal_flip=False)
split_point = 48000 # 80%/20% (48000,12000)
dg_trainer = data_generator.flow(X[:split_point],
 Y[:split_point], batch_size=28)
         
epochs = 100
history = model.fit(dg_trainer,
 epochs=epochs,
 verbose=1)

score = model.evaluate(X, Y, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'image' is not defined

In [ ]:
# summarize history for accuracy

plt.subplot(211)
plt.plot(history.history['sparse_categorical_accuracy'])
#plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('Accuracy (P)')
plt.xlabel('Epochs')
plt.legend(['Training', 'Validation'], loc='lower right')

# summarize history for loss

plt.subplot(212)
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.tight_layout()
plt.show()